In [27]:
import openpyxl
import subprocess
import requests
import xml.etree.ElementTree as ET

def get_installed_packages():
    # Use pip to retrieve the list of installed packages
    result = subprocess.run(['pip', 'list', '--format=freeze'], capture_output=True, text=True)
    output = result.stdout.strip().split('\n')
    
    # Parse the output to extract the package names and versions
    installed_packages = {}
    for line in output:
        package_name, package_version = line.split('==')
        installed_packages[package_name.lower()] = package_version
    
    return installed_packages

def get_latest_version(package_name):
    # Fetch the package information from PyPI
    response = requests.get(f'https://pypi.org/pypi/{package_name}/json')
    package_info = response.json()
    
    # Extract the latest version number, web link, and release date
    releases = package_info.get('releases', {})
    versions = list(releases.keys())
    latest_version = versions[-1] if versions else None
    
    latest_release = releases.get(latest_version, [])
    if latest_release:
        release_info = latest_release[0]
        url = release_info.get('url')
        upload_time = release_info.get('upload_time')
    else:
        url = None
        upload_time = None
    
    return latest_version, url, upload_time

# Get the installed packages and their versions
installed_packages = get_installed_packages()

# Create a new Excel workbook
workbook = openpyxl.Workbook()
sheet = workbook.active

# Write headers
sheet['A1'] = 'Package Name'
sheet['B1'] = 'Old Version'
sheet['C1'] = 'Updated Version'
sheet['D1'] = 'Web Link'
sheet['E1'] = 'Upload Time'

# Create a list to store the updated packages
updated_packages = []

# Iterate through installed packages and check for updates
for package_name, installed_version in installed_packages.items():
    latest_version, web_link, upload_time = get_latest_version(package_name)
    
    if latest_version != installed_version:
        updated_packages.append((package_name, installed_version, latest_version, web_link, upload_time))

# Sort the updated packages based on the upload time in descending order, handling None values
updated_packages.sort(key=lambda package: package[4] if package[4] else '', reverse=True)

# Write the updated packages to the Excel sheet
row = 2
for package in updated_packages:
    package_name, installed_version, latest_version, web_link, upload_time = package
    sheet.cell(row=row, column=1, value=package_name)
    sheet.cell(row=row, column=2, value=installed_version)
    sheet.cell(row=row, column=3, value=latest_version)
    sheet.cell(row=row, column=4, value=web_link)
    sheet.cell(row=row, column=5, value=upload_time)
    row += 1

# Save the workbook
workbook.save('updated_packages.xlsx')  
